In [1]:
!pip install selenium webdriver-manager

In [2]:
import os
import csv
import time
import base64
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [3]:
# Define categories
categories = [
    "cats", "dogs", "cars", "mountains", "sunsets",
    "birds", "flowers", "skyscrapers", "beaches", "forests",
    "food", "sports", "books", "bicycles", "street art",
    "technology", "musical instruments", "historical landmarks", "wild animals", "space"
]

In [4]:
# Set up Chrome WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Ensure output directories exist
os.makedirs("images", exist_ok=True)

In [5]:
# CSV file to store metadata
csv_file = open("image_metadata.csv", "w", newline="", encoding="utf-8")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["Category", "Image_URL", "Filename", "Resolution"])

40

In [6]:
# Function to download images
def download_images(category):
    search_url = f"https://www.google.com/search?tbm=isch&q={category}"
    driver.get(search_url)
    time.sleep(3)

    images = driver.find_elements(By.TAG_NAME, "img")[1:51]
    category_folder = f"images/{category}"
    os.makedirs(category_folder, exist_ok=True)

    for idx, img in enumerate(images):
        img_url = img.get_attribute("src")

        if img_url:
            filename = f"{category}_{idx}.jpg"
            filepath = os.path.join(category_folder, filename)

            if img_url.startswith("data:image"):  # Handle Base64 images
                image_data = img_url.split(",", 1)[1]  # Get the Base64 data part
                with open(filepath, "wb") as file:
                    file.write(base64.b64decode(image_data))
            else:  # Handle normal URL-based images
                try:
                    response = requests.get(img_url, timeout=5)
                    if response.status_code == 200:
                        with open(filepath, "wb") as file:
                            file.write(response.content)
                except Exception as e:
                    print(f"Skipping image {idx} for {category} due to error: {e}")
            
            csv_writer.writerow([category, img_url, filename, "N/A"])

In [7]:
# Download images for each category
for category in categories:
    download_images(category)

csv_file.close()
driver.quit()
print("Image dataset collection completed.")

Image dataset collection completed.


Dataset Name: VisualScope-20

Use Case: Can be used for training image classification models, object detection, or content-based image retrieval systems.